# AgCl as first test case

First test runs comparing tableau based solver (numerical or analytical gradients and logX versus X solver) and PHREEQC.

In [1]:
% setup environment
% set path to equilibrium solver
txt=pwd;
n=length(txt);
rootpath=txt; rootpath=txt(1:n-19);
PHREEQCpath=[rootpath,'/runPHREEQC'];
addpath(PHREEQCpath);
JEQUILIBpath=[rootpath,'/JEQUILIB'];
addpath(JEQUILIBpath);
% clear data and number format
clear; format short e
% turn off warnings
warning off
%graphics_toolkit ("notebook"); 
%graphics_toolkit ("plotly"); 
%graphics_toolkit("gnuplot")

In [2]:
% MODEL USING TABLEAU (different gradients or log versus linear variables)
%graphics_toolkit("notebook")
%model with just inorganic speciation
%geochemical model
% input the constants and solution chemistry

flag1=1; %flag1 for concentration based search vector (1) or for log based search vector (2).
flag2=2; %flag2 for numerical derivatives (2) or analtyical derivatives (1)
%database=[]; 
flag3=0; % flag3 1 for show warnings. 0 no warnings
flag4=0; %if=1 solve tableau one line at a time to get a good initial guess (no solids). really challenging probs only
flag5=0; %if=1 use stored initial guess for including solids

AgT=1e-4; ClT=1e-4; pH=7; 

TOTALS=[AgT ClT];
TOTALS(TOTALS==0)=1e-16; % get rid of zero values, b/c div by zero error
%TOTALS=[inorganicTOTALS]; %TOTALS=inorganicTOTALS;
pH=pH; pe=20.75-pH; 

tic;
[Ag,Cl,AgCls,MASSERR]=AgCltableau(pH,pe,TOTALS',flag1,flag2,flag3,flag4,flag5);
tableautime=toc;

Agmasserror=MASSERR(1); Clmasserror=MASSERR(2);

X no ITER EXEED no solid logX


In [3]:
% MODEL USING PHREEQC

minerals=[{'AgCls'}]; totalvector=[AgT; ClT; ClT]; totalnames=[{'Ag'}; {'Cl'}; {'Na'}]; speciesexport=[{'Ag+'}; {'Cl-'}];
%database=['llnl_nosolubleAgCl.dat']; 
acid=['NaOH']; show=0; T=25;
%database=['AgCl.dat']; 
database=['simplestAgCl.dat']; 

tic
[solutionspeciesconcs, speciesnames, solidconcs, solidnames]=runPHREEQCv2noHA(T,pH,pe,totalnames,totalvector,minerals,speciesexport,database,show,acid);
PHREEQCtime=toc;

AgPHREEQC=solutionspeciesconcs(1); ClPHREEQC=solutionspeciesconcs(2); AgClsPHREEQC=solidconcs(1);
AgmasserrorPHREEQC=AgT-AgPHREEQC-AgClsPHREEQC; ClmasserrorPHREEQC=ClT-ClPHREEQC-AgClsPHREEQC;

In [4]:
% exact model

Ksp=10^-9.7453;

% check SI

IAP=AgT*ClT;

Agexact=AgT; Clexact=ClT; AgClsexact=0;

if IAP>=Ksp
a=1; b=ClT-AgT; c=-Ksp;
t=roots([a b c]); t=t(t>0); t=real(t(imag(t)==0));
Agexact=t; AgClsexact=AgT-Ag; Clexact=ClT-AgClsexact;
end


In [5]:
% compare models

Agcompare=[Ag AgPHREEQC Agexact]
Clcompare=[Cl ClPHREEQC Clexact]
AgClcompare=[AgCls AgClsPHREEQC AgClsexact]
Agmasserrorcompare=[Agmasserror AgmasserrorPHREEQC]
Clmasserrorcompare=[Clmasserror ClmasserrorPHREEQC]
timecompare=[tableautime PHREEQCtime]

Agcompare,1,2,3
1,1.34076e-05,1.34076e-05,1.34076e-05


Clcompare,1,2,3
1,1.34076e-05,1.34076e-05,1.34076e-05


AgClcompare,1,2,3
1,8.65924e-05,8.65924e-05,8.65924e-05


Agmasserrorcompare,1,2
1,1.35525e-14,0


Clmasserrorcompare,1,2
1,1.35525e-14,0


timecompare,1,2
1,0.247158,0.0116689


fatal: caught signal Aborted -- stopping myself...